# Querying Vagus Structure with *ssvtools*

Subject-Specific Vagus scaffolds files can be queried using the [Zinc API](https://cmlibs.org/documentation/api/) however, because it only provides a general interface to Finite Element models, another Python library [ssvtools](https://github.com/ABI-Software/ssvtools) is provided as a layer over Zinc to work more conveniently with SPARC vagus scaffolds, and its functions are used in combination with direct Zinc API. You are encouraged to read and reuse the code for these utilities.

In the following a test vagus scaffold is loaded and its structure is queried with the ssvtools' `get_vagus_structure_maps()` function.

In [2]:
from cmlibs.zinc.context import Context
from ssvtools.query_structure import (
    evaluate_branch_start_coordinates, get_marker_data, get_vagus_structure_maps, get_vagus_trunk_group)

context = Context("SPARC-SSV")
region = context.getDefaultRegion()
result = region.readFile("resources/vagus_test_scaffold1.exf")
fieldmodule = region.getFieldmodule()
structure_map, common_branch_map = get_vagus_structure_maps(fieldmodule)

The first return value is a dict mapping from trunk/branch group names to a list [parent group name (or None), list of child group names]:

In [3]:
print(structure_map)

{'left vagus nerve': [None, ['left A thoracic cardiopulmonary branch of vagus nerve', 'left B thoracic cardiopulmonary branch of vagus nerve', 'left superior laryngeal nerve']], 'left A branch of superior laryngeal nerve': ['left superior laryngeal nerve', []], 'left A thoracic cardiopulmonary branch of vagus nerve': ['left vagus nerve', []], 'left B thoracic cardiopulmonary branch of vagus nerve': ['left vagus nerve', []], 'left superior laryngeal nerve': ['left vagus nerve', ['left A branch of superior laryngeal nerve']]}


The second return value is a map from common branch names to a list of variant branches with A, B, C, etc. in their name. The common group allows all branches innervating the same organ or structure to be visualized together (e.g. using Argon Viewer from the [SPARC Scaffold Mapping Tools](https://sparc.science/tools-and-resources/1mv8q3JckdpSYpPK9dvdKx)). Note that the common group names are not added into the structure map which exclusively contains the unique names of vagus trunk and branches specified in the source data the scaffold was generated from.

In [4]:
print(common_branch_map)

{'left branch of superior laryngeal nerve': ['left A branch of superior laryngeal nerve'], 'left thoracic cardiopulmonary branch of vagus nerve': ['left A thoracic cardiopulmonary branch of vagus nerve', 'left B thoracic cardiopulmonary branch of vagus nerve']}


Next we wish to evaluate the vagus coordinates of the branches coming immediately off the trunk i.e. excluding branches of branches. The third component of vagus coordinates is the normalized coordinate down the trunk in the range [0.0, 1.0], while components 1 and 2 range over [-0.006, +0.006] in the left, anterior direction, chosen to give the nominal epineurium diameter a 3mm:500mm aspect ratio over length.

![SSV vagus coordinates and markers](images/ssv_material_markers.png)

**Figure 1. Vagus coordinates and <font color="red">level markers</font>**

The first 3-component vector after the branch name is the vagus coordinates where the branch originates. Note that branches need not start on the centroid of the trunk, and in fact should physically start near the epineurium.

The second 3-component vector after the branch name is the direction the branch initially goes from the trunk in vagus coordinates, meaning it is compressed/stretched depending on the spacing of that subject's level markers compared to their reference positions.

These coordinates are common across all vagus scaffolds so plotting these positions and directions for all subjects will show trends across the population. Note that while it's possible to get a mean vagus coordinates of the start of all branches to a common target, especially the 3rd component down the vagus, one cannot trivially average the directions as branches on opposite sides would cancel each other. Instead, the visualization of all subjects' positions of directions needs to inform what reduced forms can represent the population average.

In [5]:
vagus_trunk_group = get_vagus_trunk_group(fieldmodule)
vagus_trunk_group_name = vagus_trunk_group.getName()
trunk_branch_list = structure_map[vagus_trunk_group_name][1]
vagus_coordinates = fieldmodule.findFieldByName("vagus coordinates")
branch_start_vagus_coordinates = evaluate_branch_start_coordinates(vagus_coordinates, trunk_branch_list)
print("Vagus coordinates:", branch_start_vagus_coordinates)

Vagus coordinates: [('left A thoracic cardiopulmonary branch of vagus nerve', [-0.0003328408968624165, 1.7493737182796838e-05, 0.3813658363063589], [-0.9106917613635638, 0.38295521752225525, 0.15487355215469165]), ('left B thoracic cardiopulmonary branch of vagus nerve', [0.0010020650864467912, -0.001554513928071552, 0.40674113258208205], [0.5645932995214857, -0.630854592172089, 0.5322188362608271]), ('left superior laryngeal nerve', [0.0008767095326078595, 0.00045882124897275354, 0.1315331326187551], [0.6662449900204912, 0.64287539363479, -0.3779270320200837])]


Next we evaluate the start straight coordinates for all the branches off the trunk, which have a more physical meaning. The z-component is the physical distance down that subject's vagus trunk, while the (x, y) components give the offset from the centroid in the (left, anterior) direction, all in the same units the original micro-CT tracings were measured in. Straight coordinates are the best starting point for fitting to in-body tracings of the same subject, as they have both physically correct dimensions and are smooth and untwisted.

In [6]:
straight_coordinates = fieldmodule.findFieldByName("straight coordinates")
branch_start_straight_coordinates = evaluate_branch_start_coordinates(straight_coordinates, trunk_branch_list)
print("Straight coordinates:", branch_start_straight_coordinates)

Straight coordinates: [('left A thoracic cardiopulmonary branch of vagus nerve', [-19.457496230805386, 1.0226637666995089, 23808.53001264119], [-0.9097938227559508, 0.38257762513656673, 0.16096447067591738]), ('left B thoracic cardiopulmonary branch of vagus nerve', [57.70817451180162, -89.52328771404899, 25364.382333258563], [0.5530901132994802, -0.6180013793215057, 0.5587178373088555]), ('left superior laryngeal nerve', [85.3419692789708, 44.66326357590322, 8624.57697803279], [0.6981983439472732, 0.6737079331530234, -0.24215014622330203])]


The remaining "coordinates" field gives the geometric configuration the scaffold was created in, which for SPARC REVA data is the excised table or virtual stitched coordinate system which the Scaffold Mapping Tools' Segmentation Stitcher plugin outputs, so the positions are not anatomically realistic. It has an import role in that it is in the same configuration as the micro-CT images transformed with the translations and rotations calculated by the Segmentation Stitcher. Hence, positions in this coordinate system can be converted into the space of each micro-CT image for that subject, and through this, it is possible to map cross-section images of the vagus to fixed material coordinates down the trunk for apples-to-apples comparison of different subjects at the same vagus material coordinate.

Next we can query the positions of level markers. The following function produces records of (marker name, marker location (element number, [element local coordinates 1, 2, 3]), marker vagus coordinates [, optional host coordinates]) for all level markers in order down the trunk. The marker points already know their vagus coordinates as they give it their permanent material locations, but one can optionally ask for a field on the host scaffold to be evaluated at their marker location e.g. straight coordinates. The vagus scaffold level marker names and their vagus coordinates are shown in Figure 1, above, for the left vagus nerve trunk.

In [5]:
marker_data = get_marker_data(fieldmodule, straight_coordinates)
print(marker_data)

[('left level of superior border of jugular foramen on the vagus nerve', (1, [0.6843240000000003, 0.5, 0.5]), [0.0, 0.0, 0.02737296], [-6.50506937191939e-14, -1.310900721441712e-14, 1774.9571795976287]), ('left level of inferior border of jugular foramen on the vagus nerve', (2, [0.1087380000000001, 0.5, 0.5]), [0.0, 0.0, 0.04434952], [1.0209665570151899e-13, 9.768198918855594e-14, 2991.9103185208987]), ('left level of angle of the mandible on the vagus nerve', (4, [0.1332685000000003, 0.5, 0.5]), [0.0, 0.0, 0.12533074], [1.4483477432177637e-14, 2.1376726263214913e-14, 8245.63333162319]), ('left level of carotid bifurcation on the vagus nerve', (4, [0.9345910000000012, 0.5, 0.5]), [0.0, 0.0, 0.15738364], [-8.404728289972044e-14, 1.2993009323710421e-14, 10174.73968280258]), ('left level of laryngeal prominence on the vagus nerve', (6, [0.1354834999999998, 0.5, 0.5]), [0.0, 0.0, 0.20541934], [3.3090283510828584e-14, -1.6217927879715146e-14, 13115.836142905777]), ('left level of superior 

In developing the vagus scaffold we callibrated the spacing of the 3rd component of the level markers' vagus coordinates to match proportionately to their average physical distance (z-component of the straight coordinates) over the population), so developing the vagus scaffolds for the SPARC REVA projects is optimized over at least 2 iterations, with separate values for left and right vagus trunks.